# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

<b>Характеристики датасета</b><p>Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
<ul style="list-style-type:square;">
<li/>сalls — количество звонков,
<li/>minutes — суммарная длительность звонков в минутах,
<li/>messages — количество sms-сообщений,
<li/>mb_used — израсходованный интернет-трафик в Мб,
<li/>is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
    </ul></p>

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
data = pd.read_csv('/datasets/users_behavior.csv')
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Файл прочитал, информация выведена на экран

## 2. Разбейте данные на выборки

In [2]:
data_train, data_check = train_test_split(data, test_size=0.4, random_state=851232)
data_valid, data_test = train_test_split(data_check, test_size=0.5, random_state=851232)

train_features = data_train.drop(columns='is_ultra')
train_answers = data_train['is_ultra']

valid_features = data_valid.drop(columns='is_ultra')
valid_answers = data_valid['is_ultra']

test_features = data_test.drop(columns='is_ultra')
test_answers = data_test['is_ultra']

Как видно, я разбил данные на три выборки, и каждую выборку разбил на признаки и целевой признак. Итого, обучающаяся выборка занимает 60% всех данных, валидационная и тестовая - по 20%.

## 3. Исследуйте модели

In [3]:
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
best_model = 0
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 60):
    for dpth in range(1, 13):
        model = RandomForestClassifier(n_estimators = est, max_depth=dpth, random_state=892364)
        model.fit(train_features, train_answers)
        predictions = model.predict(valid_features)
        result = accuracy_score(valid_answers, predictions)
        if result > best_result:
            best_result = result
            best_model = model
            best_est = est
            best_depth = dpth

print(f'Optimal estimators = {best_est}, optimal depth ={best_depth}')
print(f'Accuracy on valid_data equals to {round(best_result, 4)}')

Optimal estimators = 24, optimal depth =12
Accuracy on valid_data equals to 0.8165
CPU times: user 1min 10s, sys: 297 ms, total: 1min 10s
Wall time: 1min 10s


Ячейка выше работает минуты +-2. Но зато на тестовой выборке точность 80+%

In [5]:
from sklearn.linear_model import LogisticRegression
another_model = LogisticRegression(random_state=5213, solver='lbfgs')
another_model.fit(train_features, train_answers)
another_pred = another_model.predict(valid_features)
another_result = accuracy_score(valid_answers, another_pred)
print('linear_model accuracy =', another_result )

linear_model accuracy = 0.7200622083981337


<b> Итого</b>
<p> Мы рассмотрели модей линейной регрессии и модель случайного леса. И только модель случайного леса дала нам результат, удовлетворяющий наши условия, а именно, точность более 75%. Далее проверим лучшую модель на тестовой выборке.

## 4. Проверьте модель на тестовой выборке

In [6]:
test_pred = best_model.predict(test_features)
finish_result = accuracy_score(test_answers, test_pred)

print('Точность предсказаний на тестовой выборки равна', finish_result, 'или {:.2%}'.format(finish_result))

Точность предсказаний на тестовой выборки равна 0.80248833592535 или 80.25%


Мы справились с задачей и достигли accuracy 75+%.

## 5. (бонус) Проверьте модели на адекватность

In [10]:
# сравним с dummy classifier sklearn
from sklearn.dummy import DummyClassifier
best_dum_mod=0
best_dum_res=0
best_dum_str = 'l'
for k in ('stratified', 'most_frequent', 'prior', 'uniform'):
    dum_model = DummyClassifier(strategy=k, random_state=67)
    dum_model.fit(train_features, train_answers)
    dum_pred = dum_model.predict(valid_features)
    dum_res = accuracy_score(valid_answers, dum_pred)
    if dum_res > best_dum_res:
        best_dum_mod = dum_model
        best_dum_res = dum_res
        best_dum_str = k
print(f'Dummy_Accuracy on valid_data equals to {round(best_dum_res, 4)}')
print('\nBest Dummy strategy is', best_dum_str, '\n')

test_dum_pred = best_dum_mod.predict(test_features)
finish_dum_result = accuracy_score(test_answers, test_dum_pred)

print('Точность предсказаний dummy model на тестовой выборки равна', finish_dum_result, 'или {:.2%}'.format(finish_dum_result))
print('\nКак видно, самый так сказать тупой класссификатор выдает меньший процент правильных ответов', 
      'нежели построенный нами , {:.2%} против {:.2%} на тестовой выборке. Наша модель адекватна.'.format(best_result, finish_dum_result))

Dummy_Accuracy on valid_data equals to 0.7014

Best Dummy strategy is most_frequent 

Точность предсказаний dummy model на тестовой выборки равна 0.6796267496111975 или 67.96%

Как видно, самый так сказать тупой класссификатор выдает меньший процент правильных ответов нежели построенный нами , 81.65% против 67.96% на тестовой выборке. Наша модель адекватна.


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
